<a href="https://colab.research.google.com/github/DevNicque/MyCods/blob/master/C%C3%B3pia_de_base_atem_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Configurações

In [161]:
!pip install elasticsearch

In [162]:
import pandas as pd

import calendar
from datetime import datetime, timedelta

from elasticsearch import helpers,Elasticsearch
import json

In [163]:
elasticsearch_username = 'elastic'
elasticsearch_password = 'gxjzXnMQ9ZGHE9Huc7'
elasticsearch_host = '159.89.82.132'
es = Elasticsearch(server_hostname=[elasticsearch_host],
    http_auth=(elasticsearch_username,
    elasticsearch_password),scheme="https",
    hosts=[elasticsearch_host],port=9300,
    verify_certs=False)

/usr/local/lib/python3.6/dist-packages/elasticsearch/connection/http_urllib3.py:206: UserWarning: Connecting to https://159.89.82.132:9300 using SSL with verify_certs=False is insecure.
  % self.host


#Importação dos dados

In [165]:
df = pd.read_csv('atem.csv')
df.head()

,respondent_id,collector_id,date_created,date_modified,ip_address,email_address,first_name,last_name,custom_1,Digite os dados do pesquisado,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Qual o cargo do respondente?,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Qual o segmento?,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,"Quanto a prestação de serviço do seu assessor comercial, classifique os seguintes itens:",Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,"Em linhas gerais, como considera o relacionamento com o seu assessor comercial:",Unnamed: 38,"SE ENTREGA responder as perguntas 6 e 7Em caso de entrega do produto no posto ou empresa do cliente, classifique o serviço prestado pela ATEM nos itens abaixo:",Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Em linhas gerais como considera o serviço de entrega da ATEM?,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,SE RETIRA responder as perguntas 8 e 9Em caso de retirada do produto na base da ATEM:,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Em linhas gerais como considera o serviço de retirada do produto na base da ATEM?,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,"Quanto a Central de Atendimento da ATEM, classifique os seguintes itens:",Unnamed: 62,Unnamed: 63,"Você já fez alguma reclamação ou solicitação na CA?Se SIM, responder P12 e P13. Se NÃO pular para P14","Em caso positivo para a pergunta 10, quantas vezes já abriu chamado na central:",Quanto a sua reclamação ou solicitação.,Unnamed: 67,Em linhas gerais como considera o serviço da Central de Atendimento da ATEM:,Unnamed: 69,"(Somente posto ATEM e pontão ATEM) Quanto as ações promocionais de Marketing, você observa que:",Unnamed: 71,Unnamed: 72,Observações da Pesquisa: (Atendente)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nome,Empresa,Endereço,Endereço 2,Cidade/Município,Estado,Código postal,País,Endereço de email,Número de telefone,Diretor (a),Gerente,Coordenador (a),Assistente,Outro (especifique),ATEM,COF (Consumidor Final),COFG (Consumidor Final Governamental),POSBB (Postos Bandeira Branca),PTAT (Pontão ATEM),PTBB (pontão Bandeira Branca),TRR (Transportador Revendedor Retalista),TRRNI (Transportador Revendedor Retalista na N...,Cordialidade ( é gentil e atencioso nos atendi...,Disponibilidade ( atende fora do dia de visita...,Eficácia ( lhe entrega todas as informações e ...,Eficiência ( traz soluções adequadas para o se...,Orientações Técnicas Legais ( quando necessári...,Response,Por que? comente,Cordialidade do Motorista (ele foi gentil e at...,Entrega no Prazo (o produto foi entregue dentr...,O Ciclo do Pedido é adequado? (tempo decorrido...,Condições de Frota (o estado de manutenção pas...,Procedimentos de segurança no momento da entre...,5.0,4.0,3.0,2.0,1.0,Por que? Comente,Cordialidade do atendimento (o atendimento na ...,Mercadoria disponível no Prazo (o produto esta...,Tempo do ciclo do pedido é adequado? (entre a ...,Procedimentos de segurança no momento da entre...,Há uma comunicação de agenda eficiente?,5.0,4.0,3.0,2.0,1.0,Por que? comente.,Cordialidade ( o atendimento da atendente foi ...,Eficácia ( tempo de espera e tempo de resposta...,Eficiência ( o suporte atendeu suas solicitaçõ...,Response,Response,Recebeu retorno em até 48h?,Foi apresentada solução da sua solicitação?,Response,Por que atribuiu esta nota? Comente,há interesse dos clientes nas ações?,há aumento de vendas no período?,há conquista de novos clientes?,Open-Ended Response
1,1.227786e+10,397988367.0,12/22/2020 02:38:37 PM,12/22/2020 03:39:37 PM,177.25.139.65,NaN,NaN,NaN,NaN,YURI LEMOS,PIONEIRO COMBUSTIVEIS LTDA,NaN,NaN,MANAUS,AMAZONAS,NaN,NaN,NaN,(092) 981014422,NaN,NaN,NaN,NaN,SOCIO,NaN,COF (Consumidor Final),NaN,NaN,NaN,NaN,NaN,NaN,5,5,5,5,5,5,NADA A RECLAMAR,5,5,5,5,5,5.0,NaN,NaN,NaN,NaN,PELO TEMPO DE ENTREGA DENTRO DO HORARIO COMERCIAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NÃO,NaN,NaN,NaN

In [166]:
df.describe

<bound method NDFrame.describe of      respondent_id  ...               Observações da Pesquisa: (Atendente)
0              NaN  ...                                Open-Ended Response
1     1.227786e+10  ...                                          MIKAELLEN
2     1.227785e+10  ...       OBS: PREÇO DA GASOLINA MUITO ALTO  MIKAELLEN
3     1.227784e+10  ...                                                NaN
4     1.227784e+10  ...                                                NaN
..             ...  ...                                                ...
251   1.225661e+10  ...  P.14 - Atendimento rápido, tem eficiência no a...
252   1.225660e+10  ...                                                NaN
253   1.225658e+10  ...                                                NaN
254   1.225657e+10  ...                                                NaN
255   1.225657e+10  ...                                                NaN

[256 rows x 74 columns]>

#Tratamento dos Dados

**Tratamento da tabela geral**

In [167]:
df_geral = df.loc[:,'respondent_id':'date_created']
df_geral = df_geral.drop(0)
df_geral.index = df_geral.index.rename('ID')


df_geral['data_time'] = pd.to_datetime( df_geral['date_created'])
# Coluna 'Data'
df_geral['data'] = df_geral['data_time'].dt.strftime('%Y-%m-%d')
# Coluna 'Hora'
df_geral['hora'] = df_geral['data_time'].dt.strftime('%H:%M:%S')
df_geral = df_geral.fillna(value='2000-01-01 00:00:01')

df_geral = df_geral.loc[:,'data_time':'hora']

df_geral.head()

,data_time,data,hora
ID,,,
1,2020-12-22 14:38:37,2020-12-22,14:38:37
2,2020-12-22 14:38:42,2020-12-22,14:38:42
3,2020-12-22 11:09:59,2020-12-22,11:09:59
4,2020-12-22 14:12:22,2020-12-22,14:12:22
5,2020-12-22 13:31:31,2020-12-22,13:31:31


**Tratamento da tabela cargo**

In [168]:
df3 = df.loc[:,'Qual o cargo do respondente?':'Unnamed: 23']
df3.columns = df3.loc[0]
df3 = df3.drop(0)
df3.columns = ['diretor', 'gerente', 'coordenador', 'assistente', 'outro']
df3.index = df3.index.rename('ID')
df3 = df3.fillna(value='nao')


df3.head()

,diretor,gerente,coordenador,assistente,outro
ID,,,,,
1,nao,nao,nao,nao,SOCIO
2,Diretor (a),nao,nao,nao,nao
3,nao,Gerente,nao,nao,nao
4,nao,Gerente,nao,nao,nao
5,nao,Gerente,nao,nao,nao


**Tratamento da tabela segmento**

In [169]:
df4 = df.loc[:,'Qual o segmento?':'Unnamed: 31']
df4.columns = df4.loc[0]
df4 = df4.drop(0)

df4.columns = ['ATEM', 'COF', 'COFG', 'POSBB', 'PTAT','PTBB','TRR', 'TRRNI']
df4.index = df4.index.rename('ID')
df4 = df4.fillna(value='nao')
df4.head()

,ATEM,COF,COFG,POSBB,PTAT,PTBB,TRR,TRRNI
ID,,,,,,,,
1,nao,COF (Consumidor Final),nao,nao,nao,nao,nao,nao
2,nao,COF (Consumidor Final),nao,nao,nao,nao,nao,nao
3,nao,nao,nao,POSBB (Postos Bandeira Branca),nao,nao,nao,nao
4,nao,nao,nao,POSBB (Postos Bandeira Branca),nao,nao,nao,nao
5,nao,nao,nao,POSBB (Postos Bandeira Branca),nao,nao,nao,nao


**Tratamento da tabela Dados dos Pesquisados**

In [170]:
df2 = df.loc[:,'Digite os dados do pesquisado':'Unnamed: 18']
df2.columns = df2.loc[0]
df2 = df2.drop(0)
df2.columns = ['nome', 'empresa','endereco','endereco2','cidade','estado','codigo_postal', 'pais','email','telefone']
df2.index = df2.index.rename('ID')
df2 = df2.fillna(value='nao_informado')

df2.head()

,nome,empresa,endereco,endereco2,cidade,estado,codigo_postal,pais,email,telefone
ID,,,,,,,,,,
1,YURI LEMOS,PIONEIRO COMBUSTIVEIS LTDA,nao_informado,nao_informado,MANAUS,AMAZONAS,nao_informado,nao_informado,nao_informado,(092) 981014422
2,LUCIANO ARAUJO,MANAOS COOP DE TRAN ALT COL URB EST AM,nao_informado,nao_informado,MANAUS,AMAZONAS,nao_informado,nao_informado,nao_informado,(92) 991602421
3,CRISTIANE,MANENTI COM DE COMB LTDA,nao_informado,nao_informado,PONTES E LACERDA,MATO GROSSO,nao_informado,nao_informado,nao_informado,3266-1696
4,CRISTIANE,POSTO MANENTI COMBUSTIVEIS LTDA,nao_informado,nao_informado,PONTES E LACERDA,MATO GROSSO,nao_informado,nao_informado,nao_informado,3266-1696
5,CRISTIANE,MANENTI E MANENTI,nao_informado,nao_informado,PONTES E LACERDA,MATO GROSSO,nao_informado,nao_informado,nao_informado,3266-1696


**Incluindo campo cargo**

In [171]:
aux = 1000
df2['Cargo'] =['Diretor' if a != aux
              else 'Gerente' if b != aux
              else 'Coordenador' if c != aux
              else 'Assistente' if d != aux
              else 'Outro' if e != aux
              else 'Não Informado'
              for a,b,c,d,e in zip(df3['diretor'].to_list(), df3['gerente'].to_list(), df3['coordenador'].to_list(), df3['assistente'].to_list(), df3['outro'].to_list())]

df2.head()

,nome,empresa,endereco,endereco2,cidade,estado,codigo_postal,pais,email,telefone,Cargo
ID,,,,,,,,,,,
1,YURI LEMOS,PIONEIRO COMBUSTIVEIS LTDA,nao_informado,nao_informado,MANAUS,AMAZONAS,nao_informado,nao_informado,nao_informado,(092) 981014422,Diretor
2,LUCIANO ARAUJO,MANAOS COOP DE TRAN ALT COL URB EST AM,nao_informado,nao_informado,MANAUS,AMAZONAS,nao_informado,nao_informado,nao_informado,(92) 991602421,Diretor
3,CRISTIANE,MANENTI COM DE COMB LTDA,nao_informado,nao_informado,PONTES E LACERDA,MATO GROSSO,nao_informado,nao_informado,nao_informado,3266-1696,Diretor
4,CRISTIANE,POSTO MANENTI COMBUSTIVEIS LTDA,nao_informado,nao_informado,PONTES E LACERDA,MATO GROSSO,nao_informado,nao_informado,nao_informado,3266-1696,Diretor
5,CRISTIANE,MANENTI E MANENTI,nao_informado,nao_informado,PONTES E LACERDA,MATO GROSSO,nao_informado,nao_informado,nao_informado,3266-1696,Diretor


**Incluindo campo segmento**

In [172]:
df2['Segmento'] =['ATEM' if a != aux
              else 'COF' if b != aux
              else 'COFG' if c != aux
              else 'POSBB' if d != aux
              else 'PTAT' if e != aux
              else 'PTBB' if e != aux
              else 'TRR' if e != aux
              else 'TRRNI' if e != aux
              else 'Não Informado'
              for a,b,c,d,e,f,g,h in zip(df4['ATEM'].to_list(), df4['COF'].to_list(), df4['COFG'].to_list(), df4['POSBB'].to_list(), df4['PTAT'].to_list(), df4['PTBB'].to_list(), df4['TRR'].to_list(), df4['TRRNI'].to_list())]
               

df2.head()

,nome,empresa,endereco,endereco2,cidade,estado,codigo_postal,pais,email,telefone,Cargo,Segmento
ID,,,,,,,,,,,,
1,YURI LEMOS,PIONEIRO COMBUSTIVEIS LTDA,nao_informado,nao_informado,MANAUS,AMAZONAS,nao_informado,nao_informado,nao_informado,(092) 981014422,Diretor,ATEM
2,LUCIANO ARAUJO,MANAOS COOP DE TRAN ALT COL URB EST AM,nao_informado,nao_informado,MANAUS,AMAZONAS,nao_informado,nao_informado,nao_informado,(92) 991602421,Diretor,ATEM
3,CRISTIANE,MANENTI COM DE COMB LTDA,nao_informado,nao_informado,PONTES E LACERDA,MATO GROSSO,nao_informado,nao_informado,nao_informado,3266-1696,Diretor,ATEM
4,CRISTIANE,POSTO MANENTI COMBUSTIVEIS LTDA,nao_informado,nao_informado,PONTES E LACERDA,MATO GROSSO,nao_informado,nao_informado,nao_informado,3266-1696,Diretor,ATEM
5,CRISTIANE,MANENTI E MANENTI,nao_informado,nao_informado,PONTES E LACERDA,MATO GROSSO,nao_informado,nao_informado,nao_informado,3266-1696,Diretor,ATEM


**Dados do Pesquisado - Tabela Final**

---



In [173]:
df2 = pd.merge(df2,df_geral, on='ID', left_index=True)
df2.to_json(f'dados_do_pesquisado.json',orient="records")

df2.head()

,nome,empresa,endereco,endereco2,cidade,estado,codigo_postal,pais,email,telefone,Cargo,Segmento,data_time,data,hora
ID,,,,,,,,,,,,,,,
1,YURI LEMOS,PIONEIRO COMBUSTIVEIS LTDA,nao_informado,nao_informado,MANAUS,AMAZONAS,nao_informado,nao_informado,nao_informado,(092) 981014422,Diretor,ATEM,2020-12-22 14:38:37,2020-12-22,14:38:37
2,LUCIANO ARAUJO,MANAOS COOP DE TRAN ALT COL URB EST AM,nao_informado,nao_informado,MANAUS,AMAZONAS,nao_informado,nao_informado,nao_informado,(92) 991602421,Diretor,ATEM,2020-12-22 14:38:42,2020-12-22,14:38:42
3,CRISTIANE,MANENTI COM DE COMB LTDA,nao_informado,nao_informado,PONTES E LACERDA,MATO GROSSO,nao_informado,nao_informado,nao_informado,3266-1696,Diretor,ATEM,2020-12-22 11:09:59,2020-12-22,11:09:59
4,CRISTIANE,POSTO MANENTI COMBUSTIVEIS LTDA,nao_informado,nao_informado,PONTES E LACERDA,MATO GROSSO,nao_informado,nao_informado,nao_informado,3266-1696,Diretor,ATEM,2020-12-22 14:12:22,2020-12-22,14:12:22
5,CRISTIANE,MANENTI E MANENTI,nao_informado,nao_informado,PONTES E LACERDA,MATO GROSSO,nao_informado,nao_informado,nao_informado,3266-1696,Diretor,ATEM,2020-12-22 13:31:31,2020-12-22,13:31:31


In [174]:
f = open ('dados_do_pesquisado.json', "r")
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'atem_dados_do_pesquisado',
    'id': f"Item{cont}{i['data_time']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1

helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(255, [])

**Tratamento das Tabelas de Avaliação**

---






**Tabela Avaliação do atendimento do acessor comercial**

In [175]:
df5 = df.loc[:,'Quanto a prestação de serviço do seu assessor comercial, classifique os seguintes itens:':'Unnamed: 36']
df5.columns = df5.loc[0]
df5 = df5.drop(0)
df5.columns = ['cordialidade','disponibilidade','eficacia','eficiencia','orientacoes_tecnicas_legais']
df5.index = df5.index.rename('ID')
df5 = df5.fillna(value=1000)

df5 = pd.merge(df5,df_geral, on='ID', left_index=True)

df5.to_json(f'avaliacao_atendimento_acessorcomercial.json',orient="records")

df5.head()

,cordialidade,disponibilidade,eficacia,eficiencia,orientacoes_tecnicas_legais,data_time,data,hora
ID,,,,,,,,
1,5,5,5,5,5,2020-12-22 14:38:37,2020-12-22,14:38:37
2,5,5,5,5,5,2020-12-22 14:38:42,2020-12-22,14:38:42
3,5,5,5,5,5,2020-12-22 11:09:59,2020-12-22,11:09:59
4,5,5,5,5,5,2020-12-22 14:12:22,2020-12-22,14:12:22
5,5,5,5,5,5,2020-12-22 13:31:31,2020-12-22,13:31:31


In [176]:
f = open ('avaliacao_atendimento_acessorcomercial.json', "r")
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'atem_avaliacao_atendimento_acessorcomercial',
    'id': f"Item{cont}{i['data_time']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1

helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(255, [])

**Tabela de Avaliação em nota relacionamento com o acessor comercial**

In [177]:
df6 = df.loc[:,'Em linhas gerais, como considera o relacionamento com o seu assessor comercial:':'Unnamed: 38']
df6.columns = df6.loc[0]
df6 = df6.drop(0)

df6.columns = ['nota', 'comentario']
df6.index = df6.index.rename('ID')
df6['nota'] = df6['nota'].fillna(value=1000)
df6['comentario'] = df6['comentario'].fillna(value='nao_informado')

df6 = pd.merge(df6,df_geral, on='ID', left_index=True)

df6.to_json(f'avaliacao_nota_relacionamento_acessorcomercial.json',orient="records")

df6.head()

,nota,comentario,data_time,data,hora
ID,,,,,
1,5,NADA A RECLAMAR,2020-12-22 14:38:37,2020-12-22,14:38:37
2,5,PERFEITO,2020-12-22 14:38:42,2020-12-22,14:38:42
3,5,nao_informado,2020-12-22 11:09:59,2020-12-22,11:09:59
4,1000,nao_informado,2020-12-22 14:12:22,2020-12-22,14:12:22
5,5,nao_informado,2020-12-22 13:31:31,2020-12-22,13:31:31


In [178]:
f = open ('avaliacao_nota_relacionamento_acessorcomercial.json', "r")
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'atem_avaliacao_nota_relacionamento_acessorcomercial',
    'id': f"Item{cont}{i['data_time']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1

helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(255, [])

**Tabela Avaliação do serviço de entrega**

In [179]:
df7 = df.loc[:,'SE ENTREGA responder as perguntas 6 e 7Em caso de entrega do produto no posto ou empresa do cliente, classifique o serviço prestado pela ATEM nos itens abaixo:':'Unnamed: 43']
df7.columns = df7.loc[0]
df7 = df7.drop(0)
df7.columns = ['cordialidade_do_motorista','entrega_no_prazo','ciclo_pedido_adequado','condicoes_de_frota','procedimentos_seguranca_na_entrega']
df7.index = df7.index.rename('ID')
df7 = df7.fillna(value=1000)

df7 = pd.merge(df7,df_geral, on='ID', left_index=True)

df7.to_json(f'avaliacao_servico_entrega.json',orient="records")

df7.head()

,cordialidade_do_motorista,entrega_no_prazo,ciclo_pedido_adequado,condicoes_de_frota,procedimentos_seguranca_na_entrega,data_time,data,hora
ID,,,,,,,,
1,5,5,5,5,5,2020-12-22 14:38:37,2020-12-22,14:38:37
2,5,5,5,5,5,2020-12-22 14:38:42,2020-12-22,14:38:42
3,1000,1000,1000,1000,1000,2020-12-22 11:09:59,2020-12-22,11:09:59
4,1000,1000,1000,1000,1000,2020-12-22 14:12:22,2020-12-22,14:12:22
5,1000,1000,1000,1000,1000,2020-12-22 13:31:31,2020-12-22,13:31:31


In [180]:

f = open ('avaliacao_servico_entrega.json', "r")
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'atem_avaliacao_servico_entrega',
    'id': f"Item{cont}{i['data_time']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1

helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(255, [])

**Tratando e gerando tabela Avaliação em nota do serviço de entrega**

In [181]:
df8 = df.loc[:,'Em linhas gerais como considera o serviço de entrega da ATEM?':'Unnamed: 49']
df8.columns = df8.loc[0]
df8 = df8.drop(0)
df8.index = df8.index.rename('ID')

aux=1000
df8['Nota'] =['5' if a != aux
              else '4' if b != aux
              else '3' if c != aux
              else '2' if d != aux
              else '1' if e != aux
              else 'Não Informado'
              for a,b,c,d,e in zip(df8[5.0].to_list(), df8[4.0].to_list(), df8[3.0].to_list(), df8[2.0].to_list(), df8[1.0].to_list())]
               
df8 = df8.drop(columns=[5.0,4.0,3.0,2.0,1.0])
df8.columns = [ 'comentario','nota']
df8['nota'] = df8['nota'].fillna(value=1000)
df8['comentario'] = df8['comentario'].fillna(value='nao_informado')

df8 = pd.merge(df8,df_geral, on='ID', left_index=True)

df8.to_json(f'avaliacao_nota_servico_entrega.json',orient="records")



df8.head()

,comentario,nota,data_time,data,hora
ID,,,,,
1,PELO TEMPO DE ENTREGA DENTRO DO HORARIO COMERCIAL,5,2020-12-22 14:38:37,2020-12-22,14:38:37
2,VAI MAIS MOTORISTAS BONS QUE RUINS,5,2020-12-22 14:38:42,2020-12-22,14:38:42
3,nao_informado,5,2020-12-22 11:09:59,2020-12-22,11:09:59
4,nao_informado,5,2020-12-22 14:12:22,2020-12-22,14:12:22
5,nao_informado,5,2020-12-22 13:31:31,2020-12-22,13:31:31


In [182]:

f = open ('avaliacao_nota_servico_entrega.json', "r")
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'atem_avaliacao_nota_servico_entrega',
    'id': f"Item{cont}{i['data_time']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1

helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(255, [])

**Tabela Avaliação da retirada do produto na base da ATEM**

In [183]:
df9 = df.loc[:,'SE RETIRA responder as perguntas 8 e 9Em caso de retirada do produto na base da ATEM:':'Unnamed: 54']
df9.columns = df9.loc[0]
df9 = df9.drop(0)
df9.columns = ['cordialidade_do_atendimento','mercadoria_disponivel_no_prazo','tempo_do_ciclo_do_pedido_adequado','procedimentos_de_seguranca_na_entrega','comunicacao_de_agenda_eficiente']
df9.index = df9.index.rename('ID')
df9 = df9.fillna(value=1000)

df9 = pd.merge(df9,df_geral, on='ID', left_index=True)

df9.to_json(f'avaliacao_retirada_na_base_atem.json',orient="records")


df9.head()

,cordialidade_do_atendimento,mercadoria_disponivel_no_prazo,tempo_do_ciclo_do_pedido_adequado,procedimentos_de_seguranca_na_entrega,comunicacao_de_agenda_eficiente,data_time,data,hora
ID,,,,,,,,
1,1000,1000,1000,1000,1000,2020-12-22 14:38:37,2020-12-22,14:38:37
2,1000,1000,1000,1000,1000,2020-12-22 14:38:42,2020-12-22,14:38:42
3,3,5,5,5,5,2020-12-22 11:09:59,2020-12-22,11:09:59
4,3,5,5,5,5,2020-12-22 14:12:22,2020-12-22,14:12:22
5,3,5,5,5,5,2020-12-22 13:31:31,2020-12-22,13:31:31


In [184]:
f = open ('avaliacao_retirada_na_base_atem.json', "r")
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'atem_avaliacao_retirada_na_base_atem',
    'id': f"Item{cont}{i['data_time']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1

helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(255, [])

**Tratando e Gerando tabela Avaliação em nota da retirada do produto na base da ATEM**

In [185]:

df10 = df.loc[:,'Em linhas gerais como considera o serviço de retirada do produto na base da ATEM?':'Unnamed: 60']
df10.columns = df10.loc[0]
df10 = df10.drop(0)

df10.index = df10.index.rename('ID')

aux=1000
df10['Nota'] =['5' if a != aux
              else '4' if b != aux
              else '3' if c != aux
              else '2' if d != aux
              else '1' if e != aux
              else 'Não Informado'
              for a,b,c,d,e in zip(df10[5.0].to_list(), df10[4.0].to_list(), df10[3.0].to_list(), df10[2.0].to_list(), df10[1.0].to_list())]
               
df10 = df10.drop(columns=[5.0,4.0,3.0,2.0,1.0])
df10.columns = ['comentario','nota']
df10['nota'] = df10['nota'].fillna(value=1000)
df10['comentario'] = df10['comentario'].fillna(value='nao_informado')

df10 = pd.merge(df10,df_geral, on='ID', left_index=True)


df10.to_json(f'avaliacao_nota_retirada_na_base_atem.json',orient="records")

df10.head()

,comentario,nota,data_time,data,hora
ID,,,,,
1,nao_informado,5,2020-12-22 14:38:37,2020-12-22,14:38:37
2,nao_informado,5,2020-12-22 14:38:42,2020-12-22,14:38:42
3,nao_informado,5,2020-12-22 11:09:59,2020-12-22,11:09:59
4,nao_informado,5,2020-12-22 14:12:22,2020-12-22,14:12:22
5,nao_informado,5,2020-12-22 13:31:31,2020-12-22,13:31:31


In [186]:
f = open ('avaliacao_nota_retirada_na_base_atem.json', "r")
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'atem_avaliacao_nota_retirada_na_base_atem',
    'id': f"Item{cont}{i['data_time']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1

helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(255, [])

**Tabela Avaliação em nota central de atendimento**

In [187]:
df11 = df.loc[:,'Quanto a Central de Atendimento da ATEM, classifique os seguintes itens:':'Unnamed: 63']
df11.columns = df11.loc[0]
df11 = df11.drop(0)
df11.columns = ['cordialidade','eficacia','eficiencia']
df11.index = df11.index.rename('ID')
df11 = df11.fillna(value=1000)

df11 = pd.merge(df11,df_geral, on='ID', left_index=True)

df11.to_json(f'avaliacao_nota_atendimento_da_central.json',orient="records")

df11.head()

,cordialidade,eficacia,eficiencia,data_time,data,hora
ID,,,,,,
1,1000,1000,1000,2020-12-22 14:38:37,2020-12-22,14:38:37
2,1000,1000,1000,2020-12-22 14:38:42,2020-12-22,14:38:42
3,5,5,5,2020-12-22 11:09:59,2020-12-22,11:09:59
4,5,5,5,2020-12-22 14:12:22,2020-12-22,14:12:22
5,5,5,5,2020-12-22 13:31:31,2020-12-22,13:31:31


In [188]:
f = open ('avaliacao_nota_atendimento_da_central.json', "r")
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'atem_avaliacao_nota_atendimento_da_central',
    'id': f"Item{cont}{i['data_time']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1

helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(255, [])

**Tabela Avaliação central de atendimento**
* Aplicar tratamento no campo comentário sobre a nota - separar nota do comentario

In [158]:
df12 = df.loc[:,'Você já fez alguma reclamação ou solicitação na CA?Se SIM, responder P12 e P13. Se NÃO pular para P14':'Observações da Pesquisa: (Atendente)']
df12.columns = df12.loc[0]
df12 = df12.drop(0)
df12.columns = ['fez_alguma_reclamacao_ou_solicitacao','quantas_vezes_abriu_chamado','recebeu_retorno_em_48hs',
                'apresentaram_solucao_da_solicitacao','como_considera_servico_central','comentario_sobre_a_nota','ha_interesse_dos_clientes_nas_ações',
                'ha_aumento_de_vendas_no_período','ha_conquista_de_novos_clientes','comentario_atendente_sobre_pesquisado']
df12.index = df9.index.rename('ID')
aux = 'Por que atribuiu esta nota? Comente'

df12['como_considera_servico_central'] =['0' if x == aux
                                    else x
                                    for x in df12['como_considera_servico_central'] ]


df12['fez_alguma_reclamacao_ou_solicitacao'] = df12['fez_alguma_reclamacao_ou_solicitacao'].fillna(value='nao_informado')
df12['quantas_vezes_abriu_chamado'] = df12['quantas_vezes_abriu_chamado'].fillna(value='nao_informado')
df12['recebeu_retorno_em_48hs'] = df12['recebeu_retorno_em_48hs'].fillna(value='nao_informado')
df12['apresentaram_solucao_da_solicitacao'] = df12['apresentaram_solucao_da_solicitacao'].fillna(value='nao_informado')
df12['como_considera_servico_central'] = df12['como_considera_servico_central'].fillna(value=1000)
df12['comentario_sobre_a_nota'] = df12['comentario_sobre_a_nota'].fillna(value='nao_informado')
df12['comentario_atendente_sobre_pesquisado'] = df12['comentario_atendente_sobre_pesquisado'].fillna(value='nao_informado')

#df12 = pd.merge(df12,df_geral, on='ID', left_index=True)

#df11.to_json(f'avaliacao_central_atendimento.json',orient="records")


df12

,fez_alguma_reclamacao_ou_solicitacao,quantas_vezes_abriu_chamado,recebeu_retorno_em_48hs,apresentaram_solucao_da_solicitacao,como_considera_servico_central,comentario_sobre_a_nota,ha_interesse_dos_clientes_nas_ações,ha_aumento_de_vendas_no_período,ha_conquista_de_novos_clientes,comentario_atendente_sobre_pesquisado
ID,,,,,,,,,,
1,NÃO,nao_informado,nao_informado,nao_informado,1000,nao_informado,NaN,NaN,NaN,MIKAELLEN
2,nao_informado,nao_informado,nao_informado,nao_informado,1000,nao_informado,NaN,NaN,NaN,Cliente não tem interesse em responder.
3,nao_informado,nao_informado,nao_informado,nao_informado,1000,nao_informado,NaN,NaN,NaN,Cliente não tem interesse em responder.
4,nao_informado,nao_informado,nao_informado,nao_informado,1000,nao_informado,NaN,NaN,NaN,Cliente não tem interesse em responder.
5,SIM,Entre 2 e 5 vezes,SIM,SIM,0,5,NaN,NaN,NaN,GEOVANNA
...,...,...,...,...,...,...,...,...,...,...
178,NÃO,nao_informado,nao_informado,nao_informado,5,nao_informado,NaN,NaN,NaN,"P.14 - Atendimento rápido, tem eficiência no a..."
179,nao_informado,nao_informado,nao_informado,nao_informado,5,nao_informado,NaN,NaN,NaN,nao_informado
180,NÃO,nao_informado,nao_informado,nao_informado,1000,nao_informado,Sim,Sim,Sim,nao_informado


In [159]:
count = 0
for a,b,c in zip(df12['como_considera_servico_central'],df12['comentario_sobre_a_nota'],df12['comentario_atendente_sobre_pesquisado']):
  for s in b:
    try:
      int(s)
      a = s
    except ValueError:
      pass

  

In [160]:
df12

,fez_alguma_reclamacao_ou_solicitacao,quantas_vezes_abriu_chamado,recebeu_retorno_em_48hs,apresentaram_solucao_da_solicitacao,como_considera_servico_central,comentario_sobre_a_nota,ha_interesse_dos_clientes_nas_ações,ha_aumento_de_vendas_no_período,ha_conquista_de_novos_clientes,comentario_atendente_sobre_pesquisado
ID,,,,,,,,,,
1,NÃO,nao_informado,nao_informado,nao_informado,1000,nao_informado,NaN,NaN,NaN,MIKAELLEN
2,nao_informado,nao_informado,nao_informado,nao_informado,1000,nao_informado,NaN,NaN,NaN,Cliente não tem interesse em responder.
3,nao_informado,nao_informado,nao_informado,nao_informado,1000,nao_informado,NaN,NaN,NaN,Cliente não tem interesse em responder.
4,nao_informado,nao_informado,nao_informado,nao_informado,1000,nao_informado,NaN,NaN,NaN,Cliente não tem interesse em responder.
5,SIM,Entre 2 e 5 vezes,SIM,SIM,0,5,NaN,NaN,NaN,GEOVANNA
...,...,...,...,...,...,...,...,...,...,...
178,NÃO,nao_informado,nao_informado,nao_informado,5,nao_informado,NaN,NaN,NaN,"P.14 - Atendimento rápido, tem eficiência no a..."
179,nao_informado,nao_informado,nao_informado,nao_informado,5,nao_informado,NaN,NaN,NaN,nao_informado
180,NÃO,nao_informado,nao_informado,nao_informado,1000,nao_informado,Sim,Sim,Sim,nao_informado


**Tabela de avaliação das ações promocionais de marketing**

In [189]:
df13 = df.loc[:,'(Somente posto ATEM e pontão ATEM) Quanto as ações promocionais de Marketing, você observa que:':'Unnamed: 72']
df13.columns = df13.loc[0]
df13 = df13.drop(0)
df13 = df13.fillna(value=' nao_informado')
df13.columns = ['existe_interesses_dos_clientes_nas_acoes','existe_aumento_nas_vendas_no_periodo','existe_novos_clientes']


df13.index = df13.index.rename('ID')

df13 = pd.merge(df13,df_geral, on='ID', left_index=True)

df13.to_json(f'avaliacao_acoes_promocionais_marketing.json',orient="records")

df13.head()

,existe_interesses_dos_clientes_nas_acoes,existe_aumento_nas_vendas_no_periodo,existe_novos_clientes,data_time,data,hora
ID,,,,,,
1,nao_informado,nao_informado,nao_informado,2020-12-22 14:38:37,2020-12-22,14:38:37
2,nao_informado,nao_informado,nao_informado,2020-12-22 14:38:42,2020-12-22,14:38:42
3,Sim,Sim,Sim,2020-12-22 11:09:59,2020-12-22,11:09:59
4,Sim,Sim,Sim,2020-12-22 14:12:22,2020-12-22,14:12:22
5,Sim,Sim,Sim,2020-12-22 13:31:31,2020-12-22,13:31:31


In [190]:
f = open ('avaliacao_acoes_promocionais_marketing.json', "r")
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'atem_avaliacao_acoes_promocionais_marketing',
    'id': f"Item{cont}{i['data_time']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1


helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(255, [])